In [ ]:
# run with a julia kernel
include("Optics_in_the_length_gauge.jl")
using CairoMakie

In [11]:
using Arpack
using LinearAlgebra
using Cubature
using ProgressMeter
using Base.Threads
using Distributed
using Dierckx
using PhysicalConstants
using PhysicalConstants.CODATA2018
using Unitful
using SparseArrays
using StaticArrays



include("/Users/fernandopenaranda/Documents/Work/PostdocDonosti/Packages/Optics_in_the_length_gauge/src/jdos.jl")
include("/Users/fernandopenaranda/Documents/Work/PostdocDonosti/Packages/Optics_in_the_length_gauge/src/integration.jl")

### Bands MLG

<img src="MLG_bands.jpg" alt="Alt text" width="400"/>

### DOS

In [ ]:
xbounds =  [-2π, 2π] # adimensional
ybounds =  [0, 2π/√3] # adimensional
ωlist = collect(-10:0.1:10) # in eV
broadening = 0.025 # in eV
evals = 1e6 # number of iterations
μ = 0 # in eV

h(q) = MLG_hamiltonian(μ, q)
ω, j = dos(ham, xbounds, ybounds, collect(-10:.1:10); η = 0.025, evals = 1000000);

In [ ]:
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "dos (a.u.)")
lines!(ax, ω, j)

<img src="MLG_dos.jpg" alt="Alt text" width="400"/>

Note that the broadening introduces a finite weight at $\omega = 0$

### JDOS

In [ ]:
M = 2pi/3 # modulus of the M point
xbounds =  [0,2M]
ybounds =  [-1,1] .* 2π/√3
ωlist = collect(0:0.1:10)
broadening = 0.01 # in eV
evals = 1e6
ω, j = jdos(ham, xbounds, ybounds, ωlist; η = broadening, evals = evals);

In [ ]:
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "jdos (a.u.)")
lines!(ax, ω, j)
fig

<img src="MLG_jdos.jpg" alt="Alt text" width="400"/>

## Linear optical conductivity

In [ ]:

xbounds =  [-2π, 2π]
ybounds =  [0, 2π/√3]
ωlist = collect(0:1:10)
broadening = 0.01 # in eV
evals = 10
ν = 1
μ = 0
dirJ = :x
dirE = :x

h(q) = MLG_hamiltonian(μ, ν, q)
dh(q) = MLG_hamiltonian_derivatives(ν, q)
linear_optical_conductivity(dirJ, dirE, h, dh, xbounds, ybounds, ωlist; η = broadening, evals = evals)

